In [ ]:
import cmath
import math
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from sklearn import metrics


In [ ]:
import sympy as sym
#sym.init_printing()

In [ ]:
x = sym.symbols('x')
f = sym.Eq(sym.sinh(x) + complex(1,1),complex(0,0))
res = sym.solve([f],(x))



In [ ]:
str(res)

In [ ]:
def normal_round(n):
    if n - math.floor(n) < 0.5:
        return math.floor(n)
    return math.ceil(n)

def Convert_Mutual_Impedance(LA, LB, LC, LmAB, LmAC, LmBC):
    L1 = LA + LmAB + LmAC - LmBC
    L2 = LB + LmAB + LmBC - LmAC
    L3 = LC + LmAC + LmBC - LmAB
    return [L1, L2, L3]

def Convert_Mutual_Capacitace(CmAB, CmAC, CmBC, CA0, CB0, CC0):
    C1 = ((CmAB*CmBC + CmAC * CmBC + CmAB * CmAC) / CmBC) + CA0
    C2 = ((CmAB*CmBC + CmAC * CmBC + CmAB * CmAC) / CmAC) + CB0
    C3 = ((CmAB*CmBC + CmAC * CmBC + CmAB * CmAC) / CmAB) + CC0
    return [C1, C2, C3]


def Create_Complex(Ampli,Phase,inp='rad'):
    if (inp=='rad'):
        return complex(Ampli*math.cos(Phase),Ampli*math.sin(Phase))
    else:
        return complex(Ampli*math.cos(Phase*math.pi/180),Ampli*math.sin(Phase*math.pi/180))

def Complex_Characteristic_Impedance(R,L,G,C,w):
    return cmath.sqrt((complex(R,w*L))/(complex(G,w*C)))

def Complex_Propagation_Constant(R,L,G,C,w):
    return cmath.sqrt((complex(R,w*L))*(complex(G,w*C)))

In [ ]:
class Model_Complex():
    def __init__(self, cU1, cU2, cI1, cI2, L):
        self.cU1 = cU1
        self.cU2 = cU2
        self.cI1 = cI1
        self.cI2 = cI2
        self.L = L

class Transmission_Waves(Model_Complex):
    def __init__(self, cU1, cU2, cI1, cI2, L, cGama, cZ0):
        Model_Complex.__init__(self, cU1, cU2, cI1, cI2, L)
        self.cGama = cGama
        self.cZ0 = cZ0
    def Compute_F1(self,k):
        aux1 = self.cU1 * cmath.cosh(self.cGama * self.L * k) - self.cU2 * cmath.cosh(self.cGama * self.L * (1-k))
        aux2 = self.cI1 * cmath.sinh(self.cGama * self.L * k) + self.cI2 * cmath.sinh(self.cGama * self.L * (1-k))
        return aux1 - self.cZ0 * aux2 
    def Compute_F1_derivat(self,k):
        aux1 = self.cU1 * self.cGama * self.L * cmath.sinh(self.cGama * self.L * k) + self.cU2 * self.cGama * self.L * cmath.sinh(self.cGama * self.L * (1-k))
        aux2 = self.cI1 * self.cGama * self.L * cmath.cosh(self.cGama * self.L * k) - self.cI2 * self.cGama * self.L * cmath.cosh(self.cGama * self.L * (1-k))
        return aux1 - self.cZ0 * aux2
    def Compute_F1_derivat_ordin2(self,k):
        return self.Compute_F1(k) * self.cGama * self.L * self.cGama * self.L
    def Compute_F2(self,k):
        aux1 = self.cU1 * cmath.sinh(self.cGama * self.L * k) - self.cU2 * cmath.sinh(self.cGama * self.L * (1-k))
        aux2 = self.cI1 * cmath.cosh(self.cGama * self.L * k) - self.cI2 * cmath.cosh(self.cGama * self.L * (1-k))
        return aux1 - self.cZ0 * aux2
    def Compute_F2_derivat(self,k):
        aux1 = self.cU1 * self.cGama * self.L * cmath.cosh(self.cGama * self.L * k) + self.cU2 * self.cGama * self.L * cmath.cosh(self.cGama * self.L * (1-k))
        aux2 = self.cI1 * self.cGama * self.L * cmath.sinh(self.cGama * self.L * k) + self.cI2 * self.cGama * self.L * cmath.sinh(self.cGama * self.L * (1-k))
        return aux1 - self.cZ0 * aux2
    def Compute_F2_derivat_ordin2(self,k):
        return self.Compute_F2(k) * self.cGama * self.L * self.cGama * self.L
    def Newton_Methode_F1(self,N=20):
        xn = 1
        for i in range(N+1):
            #xn1 = xn - abs((self.Compute_F1(xn)))/abs((self.Compute_F1_derivat(xn)))
            xn1 = xn - (self.Compute_F1(xn))/(self.Compute_F1_derivat(xn))
            xn = xn1
        return xn1
    def Newton_Methode_F2(self,N=20):
        xn = 1
        for i in range(N+1):
            #xn1 = xn - abs((self.Compute_F2(xn)))/abs((self.Compute_F2_derivat(xn)))
            xn1 = xn - (self.Compute_F2(xn))/(self.Compute_F2_derivat(xn))
            xn = xn1
        return xn1
    def Halley_Methode_F1(self,N=20):
        xn = 1
        for i in range(N+1):
            radical = self.Compute_F1_derivat(xn)*self.Compute_F1_derivat(xn) - 2 * self.Compute_F1_derivat_ordin2(xn)* self.Compute_F1(xn)
            #xn1 = xn - abs((2*self.Compute_F1(xn)))/abs((self.Compute_F1_derivat(xn) - cmath.sqrt(radical)))
            xn1 = xn - (2*self.Compute_F1(xn))/(self.Compute_F1_derivat(xn) - cmath.sqrt(radical))
            xn = xn1
        return xn1
    def Halley_Methode_F2(self,N=20):
        xn = 1
        for i in range(N+1):
            radical = self.Compute_F2_derivat(xn)*self.Compute_F2_derivat(xn) - 2 * self.Compute_F2_derivat_ordin2(xn)* self.Compute_F2(xn)
            xn1 = xn - abs((2*self.Compute_F2(xn))/(self.Compute_F2_derivat(xn) - cmath.sqrt(radical)))
            xn = xn1
        return xn1
    def Determin_Minim_F1_list(self, N):
        result = list()
        for i in range(N+1):
            value = abs(self.Compute_F1(i/(N+1)))
            result.append([i,i/N,value])
        minim = result[0]
        for i in range(len(result)):
            if result[i][2]<minim[2]:
                minim = result[i]
        return result
    def Determin_Minim_F2_list(self, N):
        result = list()
        for i in range(N+1):
            value = abs(self.Compute_F2(i/(N+1)))
            result.append([i,i/N,value])
        minim = result[0]
        for i in range(len(result)):
            if result[i][2]<minim[2]:
                minim = result[i]
        return result       
    def Determin_Minim_F1(self, N):
        result = list()
        for i in range(N+1):
            value = abs(self.Compute_F1(i/(N+1)))
            result.append([i,i/N,value])
        minim = result[0]
        for i in range(len(result)):
            if result[i][2]<minim[2]:
                minim = result[i]
        return minim[1]
    def Determin_Minim_F2(self, N):
        result = list()
        for i in range(N+1):
            value = abs(self.Compute_F2(i/(N+1)))
            result.append([i,i/N,value])
        minim = result[0]
        for i in range(len(result)):
            if result[i][2]<minim[2]:
                minim = result[i]
        return minim[1]    
    def Bisect_Methode_F1(self,error=0.01):
        lim_inf=0
        lim_sup=1
        result=-1
        if (self.Compute_F1(lim_inf)*self.Compute_F1(lim_sup)<=complex(0,0)):
            while (lim_sup-lim_inf>=error):
                mediu=(lim_inf+lim_sup)/2
                if (self.Compute_F1(lim_inf)*self.Compute_F1(mediu)<=complex(0,0)):
                    lim_sup=mediu
                elif (self.Compute_F1(mediu)*self.Compute_F1(lim_sup)<=complex(0,0)):
                    lim_inf=mediu
                result = (lim_inf+lim_sup)/2
        return result
    def Bisect_Methode_F2(self,error=0.01):
        lim_inf=0
        lim_sup=1
        result=-1
        if (self.Compute_F2(lim_inf)*self.Compute_F2(lim_sup)<=complex(0,0)):
            while (lim_sup-lim_inf>=error):
                mediu=(lim_inf+lim_sup)/2
                if (self.Compute_F2(lim_inf)*self.Compute_F2(mediu)<=complex(0,0)):
                    lim_sup=mediu
                elif (self.Compute_F2(mediu)*self.Compute_F2(lim_sup)<=complex(0,0)):
                    lim_inf=mediu
                result = (lim_inf+lim_sup)/2
        return result
    
    def F1_sympy_localization(self):
        k = sym.symbols('k')
        
        f = sym.Eq((self.cU1 * sym.cosh(self.cGama * self.L * k) - self.cU2 * sym.cosh(self.cGama * self.L * (1-k))) 
                   - self.cZ0 * (self.cI1 * sym.sinh(self.cGama * self.L * k) + self.cI2 * sym.sinh(self.cGama * self.L * (1-k))),complex(0,0))
        res = sym.solve([f],(k))
        return str(res)
    def F2_sympy_localization(self):
        k = sym.symbols('k')
        
        f = sym.Eq((self.cU1 * sym.sinh(self.cGama * self.L * k) - self.cU2 * sym.sinh(self.cGama * self.L * (1-k))) - 
                   self.cZ0 * (self.cI1 * sym.cosh(self.cGama * self.L * k) - self.cI2 * sym.cosh(self.cGama * self.L * (1-k))),complex(0,0))
        res = sym.solve([f],(k))
        return str(res)   
    

In [ ]:
class Cuadripols():
    def __init__(self, R, L, G, C,w):
        self.R=R
        self.L=L
        self.G=G
        self.C=C
        self.w=w
        self.Z=complex(self.R,self.w*self.L)
        self.Y=complex(self.G/2,self.w*self.C/2)
        self.elA= 1 + self.Y * self.Z
        self.elB = self.Z
        self.elC = 2 * self.Y + self.Y * self.Y * self.Z
        self.elD= 1 + self.Y * self.Z
        self.matrix = [[self.elA,self.elB],[self.elC,self.elD]]

class Cuadripol_Method(Model_Complex):
    def __init__(self, cU1, cU2, cI1, cI2, L, cMatA, Rsc):
        Model_Complex.__init__(self, cU1, cU2, cI1, cI2, L)
        self.cMatA = cMatA
        self.Rsc = Rsc
        self.cMatAsc = [[1,0],[1/self.Rsc,1]]
    def Solve_F1_sympy_localization(self, N):
        mat_input = [self.cU1, self.cI1]
        mat_output = [self.cU2, self.cI2]
        k = sym.symbols('k')
        matrix =  mat_input - np.matmul(np.matmul(np.linalg.matrix_power(self.cMatA,k),self.cMatAsc), np.matmul(np.linalg.matrix_power(self.cMatA,N-k),mat_output))        
        
        f = sym.Eq(matrix[0],complex(0,0))
        res = sym.solve([f],(k))
        return str(res)        
    def Solve_F2_sympy_localization(self, N):
        mat_input = [self.cU1, self.cI1]
        mat_output = [self.cU2, self.cI2]
        k = sym.symbols('k')
        matrix =  mat_input - np.matmul(np.matmul(np.linalg.matrix_power(self.cMatA,k),self.cMatAsc), np.matmul(np.linalg.matrix_power(self.cMatA,N-k),mat_output))        
        
        f = sym.Eq(matrix[1],complex(0,0))
        res = sym.solve([f],(k))
        return str(res)         
    def Determin_Minim_F1(self, N):
        mat_input = [self.cU1, self.cI1]
        mat_output = [self.cU2, self.cI2]
        result = list()
        for i in range(1,N + 1):
            result.append([i,i/N,abs(self.Solve(i, N)[0])])
        minim = result[0]
        for i in range(len(result)):
            if result[i][2]<minim[2]:
                minim = result[i]
        return minim[1]      
    def Determin_Minim_F2(self, N):
        mat_input = [self.cU1, self.cI1]
        mat_output = [self.cU2, self.cI2]
        result = list()
        for i in range(1,N + 1):
            result.append([i,i/N,abs(self.Solve(i, N)[1])])
        minim = result[1]
        for i in range(len(result)):
            if result[i][2]<minim[2]:
                minim = result[i]
        return minim[1]          
    def Solve(self,k, N):
        mat_input = [self.cU1, self.cI1]
        mat_output = [self.cU2, self.cI2]
        return mat_input - np.matmul(np.matmul(np.linalg.matrix_power(self.cMatA,k),self.cMatAsc), np.matmul(np.linalg.matrix_power(self.cMatA,N-k),mat_output))
    def Bisect_Methode(self, N,error=1):
        lim_inf=1
        lim_sup=N
        result=-1
        if (np.multiply(self.Solve(lim_inf),self.Solve(lim_sup))<=[complex(0,0),complex(0,0)]):
            while (lim_sup-lim_inf>=error):
                mediu=normal_round((lim_inf+lim_sup)/2)
                if (np.multiply(self.Solve(lim_inf, N),self.Solve(mediu, N))<=[complex(0,0),complex(0,0)]):
                    lim_sup=mediu
                elif (np.multiply(self.Solve(mediu, N),self.Solve(lim_sup, N))<=[complex(0,0),complex(0,0)]):
                    lim_inf=mediu
                result = (lim_inf+lim_sup)/2
        return result

In [ ]:
data_normal = pd.read_excel('ComputeFaultLocation.xlsx',sheet_name='version_1')

In [ ]:
# Z1 = Z - Zm
# Z0 = Z + 2Zm
# Z = 1/3(Z0 + 2*Z1)
# Zm = 1/3(Z0 - Z1)
data_normal["L_val"]=0
data_normal["L_mut"]=0

data_normal["C_val"]=0
data_normal["C_mut"]=0

data_normal["R_val"]=0
data_normal["R_mut"]=0

data_normal["L_1_H/km"]=0
data_normal["L_2_H/km"]=0
data_normal["L_3_H/km"]=0

data_normal["C_1_F/km"]=0
data_normal["C_2_F/km"]=0
data_normal["C_3_F/km"]=0

data_normal["R_1_Ohm/km"]=0
data_normal["R_2_Ohm/km"]=0
data_normal["R_3_Ohm/km"]=0

In [ ]:
data_normal["SD_L_1_H/km"]=0
data_normal["SD_L_2_H/km"]=0
data_normal["SD_L_3_H/km"]=0

data_normal["SD_C_1_F/km"]=0
data_normal["SD_C_2_F/km"]=0
data_normal["SD_C_3_F/km"]=0

data_normal["SD_R_1_Ohm/km"]=0
data_normal["SD_R_2_Ohm/km"]=0
data_normal["SD_R_3_Ohm/km"]=0

data_normal["SH_L_1_H/km"]=0
data_normal["SH_L_2_H/km"]=0
data_normal["SH_L_3_H/km"]=0

data_normal["SH_C_1_F/km"]=0
data_normal["SH_C_2_F/km"]=0
data_normal["SH_C_3_F/km"]=0

data_normal["SH_R_1_Ohm/km"]=0
data_normal["SH_R_2_Ohm/km"]=0
data_normal["SH_R_3_Ohm/km"]=0

In [ ]:
data_normal["val_L_1_H/km"]=0
data_normal["val_L_2_H/km"]=0
data_normal["val_L_3_H/km"]=0

data_normal["val_C_1_F/km"]=0
data_normal["val_C_2_F/km"]=0
data_normal["val_C_3_F/km"]=0

data_normal["val_R_1_Ohm/km"]=0
data_normal["val_R_2_Ohm/km"]=0
data_normal["val_R_3_Ohm/km"]=0

In [ ]:
# Z1 = Z - Zm
# Z0 = Z + 2Zm
# Z = 1/3(Z0 + 2*Z1)
# Zm = 1/3(Z0 - Z1)
for index in data_normal.index.to_list():
    if data_normal.at[index,"Tip Input"]=="(1,2)":
        data_normal.at[index, "SD_R_1_Ohm/km"] = data_normal.at[index, "Line_Parameters_R1_Ohm/km"]
        data_normal.at[index, "SH_R_1_Ohm/km"] = data_normal.at[index, "Line_Parameters_R0_Ohm/km"]
        data_normal.at[index, "SD_R_2_Ohm/km"] = data_normal.at[index, "Line_Parameters_R1_Ohm/km"]
        data_normal.at[index, "SH_R_2_Ohm/km"] = data_normal.at[index, "Line_Parameters_R0_Ohm/km"]        
        data_normal.at[index, "SD_R_3_Ohm/km"] = data_normal.at[index, "Line_Parameters_R1_Ohm/km"]
        data_normal.at[index, "SH_R_3_Ohm/km"] = data_normal.at[index, "Line_Parameters_R0_Ohm/km"]  
        
        data_normal.at[index, "SD_L_1_H/km"] = data_normal.at[index, "Line_Parameters_L1_H/km"]
        data_normal.at[index, "SH_L_1_H/km"] = data_normal.at[index, "Line_Parameters_L0_H/km"]   
        data_normal.at[index, "SD_L_2_H/km"] = data_normal.at[index, "Line_Parameters_L1_H/km"]
        data_normal.at[index, "SH_L_2_H/km"] = data_normal.at[index, "Line_Parameters_L0_H/km"] 
        data_normal.at[index, "SD_L_3_H/km"] = data_normal.at[index, "Line_Parameters_L1_H/km"]
        data_normal.at[index, "SH_L_3_H/km"] = data_normal.at[index, "Line_Parameters_L0_H/km"] 
           
        data_normal.at[index, "SD_C_1_F/km"] = data_normal.at[index, "Line_Parameters_C1_H/km"]
        data_normal.at[index, "SH_C_1_F/km"] = data_normal.at[index, "Line_Parameters_C0_H/km"]          
        data_normal.at[index, "SD_C_2_F/km"] = data_normal.at[index, "Line_Parameters_C1_H/km"]
        data_normal.at[index, "SH_C_2_F/km"] = data_normal.at[index, "Line_Parameters_C0_H/km"]           
        data_normal.at[index, "SD_C_3_F/km"] = data_normal.at[index, "Line_Parameters_C1_H/km"]
        data_normal.at[index, "SH_C_3_F/km"] = data_normal.at[index, "Line_Parameters_C0_H/km"]         
        
        data_normal.at[index, "R_val"] = (1/3) * (data_normal.at[index, "Line_Parameters_R0_Ohm/km"] 
                                            + 2 * data_normal.at[index, "Line_Parameters_R1_Ohm/km"])
        data_normal.at[index, "R_mut"] = (1/3) * (data_normal.at[index, "Line_Parameters_R0_Ohm/km"] 
                                            - data_normal.at[index, "Line_Parameters_R1_Ohm/km"])
        
        data_normal.at[index, "L_val"] = (1/3) * (data_normal.at[index, "Line_Parameters_L0_H/km"] 
                                            + 2 * data_normal.at[index, "Line_Parameters_L1_H/km"])
        data_normal.at[index, "L_mut"] = (1/3) * (data_normal.at[index, "Line_Parameters_L0_H/km"] 
                                            - data_normal.at[index, "Line_Parameters_L1_H/km"])      
        
        #data_normal.at[index, "C_val"] = (1/3) * (data_normal.at[index, "Line_Parameters_C0_H/km"] + 2 * data_normal.at[index, "Line_Parameters_C1_H/km"])
        #data_normal.at[index, "C_mut"] = (1/3) * (data_normal.at[index, "Line_Parameters_C0_H/km"] - data_normal.at[index, "Line_Parameters_C1_H/km"])  
        
        data_normal.at[index, "C_val"] = (1/3) * (data_normal.at[index, "Line_Parameters_C0_H/km"] * 2  * data_normal.at[index, "Line_Parameters_C1_H/km"]) / (data_normal.at[index, "Line_Parameters_C0_H/km"] + 2 * data_normal.at[index, "Line_Parameters_C1_H/km"])
        data_normal.at[index, "C_mut"] = (1/3) * (data_normal.at[index, "Line_Parameters_C0_H/km"] * data_normal.at[index, "Line_Parameters_C1_H/km"]) /(data_normal.at[index, "Line_Parameters_C0_H/km"] - data_normal.at[index, "Line_Parameters_C1_H/km"]) 
        
        data_normal.at[index, "R_1_Ohm/km"] = data_normal.at[index, "R_val"]
        data_normal.at[index, "R_2_Ohm/km"] = data_normal.at[index, "R_val"]
        data_normal.at[index, "R_3_Ohm/km"] = data_normal.at[index, "R_val"]
        auxL = Convert_Mutual_Impedance(data_normal.at[index, "L_val"], data_normal.at[index, "L_val"], data_normal.at[index, "L_val"], 
                                        data_normal.at[index, "L_mut"], data_normal.at[index, "L_mut"], data_normal.at[index, "L_mut"])
        data_normal.at[index, "L_1_H/km"] = auxL[0]
        data_normal.at[index, "L_2_H/km"] = auxL[1]
        data_normal.at[index, "L_3_H/km"] = auxL[2]
        auxC = Convert_Mutual_Capacitace(data_normal.at[index, "C_mut"], data_normal.at[index, "C_mut"], data_normal.at[index, "C_mut"], 
                                         data_normal.at[index, "C_val"], data_normal.at[index, "C_val"], data_normal.at[index, "C_val"])
        data_normal.at[index, "C_1_F/km"] = auxC[0]
        data_normal.at[index, "C_2_F/km"] = auxC[1]
        data_normal.at[index, "C_3_F/km"] = auxC[2]
        
        data_normal.at[index, "val_R_1_Ohm/km"] = data_normal.at[index, "R_1_Ohm/km"]
        data_normal.at[index, "val_R_2_Ohm/km"] = data_normal.at[index, "R_2_Ohm/km"]
        data_normal.at[index, "val_R_3_Ohm/km"] = data_normal.at[index, "R_3_Ohm/km"]
        data_normal.at[index, "val_L_1_H/km"] = data_normal.at[index, "L_val"]
        data_normal.at[index, "val_L_2_H/km"] = data_normal.at[index, "L_val"]
        data_normal.at[index, "val_L_3_H/km"] = data_normal.at[index, "L_val"]
        data_normal.at[index, "val_C_1_F/km"] = data_normal.at[index, "C_val"]
        data_normal.at[index, "val_C_2_F/km"] = data_normal.at[index, "C_val"]
        data_normal.at[index, "val_C_3_F/km"] = data_normal.at[index, "C_val"]        
    elif data_normal.at[index,"Tip Input"]=="(3,3)":
        data_normal.at[index, "R_1_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_11"]
        data_normal.at[index, "R_2_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_22"]
        data_normal.at[index, "R_3_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_33"]   
        
        auxL = Convert_Mutual_Impedance(data_normal.at[index, "Line_Parameters_3x3_L_11"], 
                                        data_normal.at[index, "Line_Parameters_3x3_L_22"], 
                                        data_normal.at[index, "Line_Parameters_3x3_L_33"], 
                                        data_normal.at[index, "Line_Parameters_3x3_L_12"], 
                                        data_normal.at[index, "Line_Parameters_3x3_L_13"], 
                                        data_normal.at[index, "Line_Parameters_3x3_L_23"])
        
        data_normal.at[index, "L_1_H/km"] = auxL[0]
        data_normal.at[index, "L_2_H/km"] = auxL[1]
        data_normal.at[index, "L_3_H/km"] = auxL[2]
        
        auxC = Convert_Mutual_Capacitace(data_normal.at[index, "Line_Parameters_3x3_C_12"], 
                                         data_normal.at[index, "Line_Parameters_3x3_C_13"], 
                                         data_normal.at[index, "Line_Parameters_3x3_C_23"], 
                                         data_normal.at[index, "Line_Parameters_3x3_C_11"], 
                                         data_normal.at[index, "Line_Parameters_3x3_C_22"], 
                                         data_normal.at[index, "Line_Parameters_3x3_C_33"])
        
        data_normal.at[index, "C_1_F/km"] = auxC[0]
        data_normal.at[index, "C_2_F/km"] = auxC[1]
        data_normal.at[index, "C_3_F/km"] = auxC[2]   
        
        data_normal.at[index, "SD_R_1_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_11"]
        data_normal.at[index, "SH_R_1_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_11"]
        data_normal.at[index, "SD_R_2_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_22"]
        data_normal.at[index, "SH_R_2_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_22"]        
        data_normal.at[index, "SD_R_3_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_33"]
        data_normal.at[index, "SH_R_3_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_33"]          
        
        data_normal.at[index, "SD_L_1_H/km"] = data_normal.at[index, "Line_Parameters_3x3_L_11"]  - data_normal.at[index, "Line_Parameters_3x3_L_12"]- data_normal.at[index, "Line_Parameters_3x3_L_13"]+ data_normal.at[index, "Line_Parameters_3x3_L_23"]
        data_normal.at[index, "SH_L_1_H/km"] = data_normal.at[index, "Line_Parameters_3x3_L_11"]    + 2 * data_normal.at[index, "Line_Parameters_3x3_L_12"]+ 2 * data_normal.at[index, "Line_Parameters_3x3_L_13"]- 2 * data_normal.at[index, "Line_Parameters_3x3_L_23"]
                    
        data_normal.at[index, "SD_L_2_H/km"] = data_normal.at[index, "Line_Parameters_3x3_L_22"] - data_normal.at[index, "Line_Parameters_3x3_L_12"]  + data_normal.at[index, "Line_Parameters_3x3_L_13"] - data_normal.at[index, "Line_Parameters_3x3_L_23"]        
        data_normal.at[index, "SH_L_2_H/km"] = data_normal.at[index, "Line_Parameters_3x3_L_22"] + 2 * data_normal.at[index, "Line_Parameters_3x3_L_12"]- 2 * data_normal.at[index, "Line_Parameters_3x3_L_13"] + 2 * data_normal.at[index, "Line_Parameters_3x3_L_23"] 
                    
        data_normal.at[index, "SD_L_3_H/km"] = data_normal.at[index, "Line_Parameters_3x3_L_33"]  + data_normal.at[index, "Line_Parameters_3x3_L_12"] - data_normal.at[index, "Line_Parameters_3x3_L_13"]- data_normal.at[index, "Line_Parameters_3x3_L_23"]        
        data_normal.at[index, "SH_L_3_H/km"] = data_normal.at[index, "Line_Parameters_3x3_L_33"]  - 2 * data_normal.at[index, "Line_Parameters_3x3_L_12"] + 2 * data_normal.at[index, "Line_Parameters_3x3_L_13"] + 2 * data_normal.at[index, "Line_Parameters_3x3_L_23"]    
        c1m = data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_13"] * data_normal.at[index, "Line_Parameters_3x3_C_23"]/( 0 + data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_13"]  - data_normal.at[index, "Line_Parameters_3x3_C_13"] * data_normal.at[index, "Line_Parameters_3x3_C_23"]- data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_23"])
        
        c2m = data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_13"] * data_normal.at[index, "Line_Parameters_3x3_C_23"]/ ( 0 - data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_13"] - data_normal.at[index, "Line_Parameters_3x3_C_13"] * data_normal.at[index, "Line_Parameters_3x3_C_23"] + data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_23"])
        
        c3m = data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_13"] * data_normal.at[index, "Line_Parameters_3x3_C_23"]/( 0 - data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_13"]  + data_normal.at[index, "Line_Parameters_3x3_C_13"] * data_normal.at[index, "Line_Parameters_3x3_C_23"]- data_normal.at[index, "Line_Parameters_3x3_C_12"] * data_normal.at[index, "Line_Parameters_3x3_C_23"])
        
        data_normal.at[index, "SD_C_1_F/km"] = data_normal.at[index, "Line_Parameters_3x3_C_11"] *c1m /(data_normal.at[index, "Line_Parameters_3x3_C_11"]-c1m)
        data_normal.at[index, "SH_C_1_F/km"] = data_normal.at[index, "Line_Parameters_3x3_C_11"] * 2 * c1m /(data_normal.at[index, "Line_Parameters_3x3_C_11"]+ 2 * c1m)        
        data_normal.at[index, "SD_C_2_F/km"] = data_normal.at[index, "Line_Parameters_3x3_C_22"] *c2m /(data_normal.at[index, "Line_Parameters_3x3_C_22"]-c2m)
        data_normal.at[index, "SH_C_2_F/km"] = data_normal.at[index, "Line_Parameters_3x3_C_22"] * 2 * c2m /(data_normal.at[index, "Line_Parameters_3x3_C_22"]+ 2 * c2m)          
        data_normal.at[index, "SD_C_3_F/km"] = data_normal.at[index, "Line_Parameters_3x3_C_33"] *c3m /(data_normal.at[index, "Line_Parameters_3x3_C_33"]-c3m)
        data_normal.at[index, "SH_C_3_F/km"] = data_normal.at[index, "Line_Parameters_3x3_C_33"] * 2 * c3m /(data_normal.at[index, "Line_Parameters_3x3_C_33"]+ 2 * c3m)    
        
        data_normal.at[index, "val_R_1_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_11"]
        data_normal.at[index, "val_R_2_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_22"]
        data_normal.at[index, "val_R_3_Ohm/km"] = data_normal.at[index, "Line_Parameters_3x3_R_33"]
        data_normal.at[index, "val_L_1_H/km"] = (1/3)* (data_normal.at[index, "SD_L_1_H/km"] + 2* data_normal.at[index, "SH_L_1_H/km"])
        data_normal.at[index, "val_L_2_H/km"] = (1/3)* (data_normal.at[index, "SD_L_2_H/km"] + 2* data_normal.at[index, "SH_L_2_H/km"])
        data_normal.at[index, "val_L_3_H/km"] = (1/3)* (data_normal.at[index, "SD_L_3_H/km"] + 2* data_normal.at[index, "SH_L_3_H/km"])
        data_normal.at[index, "val_C_1_F/km"] = (1/3)* (2 * data_normal.at[index, "SD_C_1_F/km"] * data_normal.at[index, "SH_C_1_F/km"]) / (data_normal.at[index, "SH_C_1_F/km"] + 2* data_normal.at[index, "SD_C_1_F/km"])
        data_normal.at[index, "val_C_2_F/km"] = (1/3)* (2 * data_normal.at[index, "SD_C_2_F/km"] * data_normal.at[index, "SH_C_2_F/km"]) / (data_normal.at[index, "SH_C_2_F/km"] + 2* data_normal.at[index, "SD_C_2_F/km"])
        data_normal.at[index, "val_C_3_F/km"] = (1/3)* (2 * data_normal.at[index, "SD_C_3_F/km"] * data_normal.at[index, "SH_C_3_F/km"]) / (data_normal.at[index, "SH_C_3_F/km"] + 2* data_normal.at[index, "SD_C_3_F/km"])

In [ ]:

data_normal["SD_Z1_val"] = 0
data_normal["SD_Z1_angle"] = 0
data_normal["SD_Z2_val"] = 0
data_normal["SD_Z2_angle"] = 0
data_normal["SD_Z3_val"] = 0
data_normal["SD_Z3_angle"] = 0
data_normal["SD_Gama1_val"] = 0
data_normal["SD_Gama1_angle"] = 0
data_normal["SD_Gama2_val"] = 0
data_normal["SD_Gama2_angle"] = 0
data_normal["SD_Gama3_val"] = 0
data_normal["SD_Gama3_angle"] = 0

data_normal["SH_Z1_val"] = 0
data_normal["SH_Z1_angle"] = 0
data_normal["SH_Z2_val"] = 0
data_normal["SH_Z2_angle"] = 0
data_normal["SH_Z3_val"] = 0
data_normal["SH_Z3_angle"] = 0
data_normal["SH_Gama1_val"] = 0
data_normal["SH_Gama1_angle"] = 0
data_normal["SH_Gama2_val"] = 0
data_normal["SH_Gama2_angle"] = 0
data_normal["SH_Gama3_val"] = 0
data_normal["SH_Gama3_angle"] = 0

data_normal["val_Z1_val"] = 0
data_normal["val_Z1_angle"] = 0
data_normal["val_Z2_val"] = 0
data_normal["val_Z2_angle"] = 0
data_normal["val_Z3_val"] = 0
data_normal["val_Z3_angle"] = 0
data_normal["val_Gama1_val"] = 0
data_normal["val_Gama1_angle"] = 0
data_normal["val_Gama2_val"] = 0
data_normal["val_Gama2_angle"] = 0
data_normal["val_Gama3_val"] = 0
data_normal["val_Gama3_angle"] = 0
for index in data_normal.index.to_list():
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "SD_R_1_Ohm/km"],
                                                     data_normal.at[index, "SD_L_1_H/km"],
                                                     0,
                                                     data_normal.at[index, "SD_C_1_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "SD_Z1_val"] = abs(aux)
    data_normal.at[index, "SD_Z1_angle"] = np.angle(aux)    
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "SD_R_2_Ohm/km"],
                                                     data_normal.at[index, "SD_L_2_H/km"],
                                                     0,
                                                     data_normal.at[index, "SD_C_2_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "SD_Z2_val"] = abs(aux)
    data_normal.at[index, "SD_Z2_angle"] = np.angle(aux)     
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "SD_R_3_Ohm/km"],
                                                     data_normal.at[index, "SD_L_3_H/km"],
                                                     0,
                                                     data_normal.at[index, "SD_C_3_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "SD_Z3_val"] = abs(aux)
    data_normal.at[index, "SD_Z3_angle"] = np.angle(aux)       

    aux = Complex_Characteristic_Impedance(data_normal.at[index, "SH_R_1_Ohm/km"],
                                                     data_normal.at[index, "SH_L_1_H/km"],
                                                     0,
                                                     data_normal.at[index, "SH_C_1_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "SH_Z1_val"] = abs(aux)
    data_normal.at[index, "SH_Z1_angle"] = np.angle(aux)    
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "SH_R_2_Ohm/km"],
                                                     data_normal.at[index, "SH_L_2_H/km"],
                                                     0,
                                                     data_normal.at[index, "SH_C_2_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "SH_Z2_val"] = abs(aux)
    data_normal.at[index, "SH_Z2_angle"] = np.angle(aux)     
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "SH_R_3_Ohm/km"],
                                                     data_normal.at[index, "SH_L_3_H/km"],
                                                     0,
                                                     data_normal.at[index, "SH_C_3_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "SH_Z3_val"] = abs(aux)
    data_normal.at[index, "SH_Z3_angle"] = np.angle(aux)   
    
    aux = Complex_Propagation_Constant(data_normal.at[index, "SD_R_1_Ohm/km"],
                                                         data_normal.at[index, "SD_L_1_H/km"],
                                                         0,
                                                         data_normal.at[index, "SD_C_1_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "SD_Gama1_val"] = abs(aux)
    data_normal.at[index, "SD_Gama1_angle"] = np.angle(aux)    
    aux = Complex_Propagation_Constant(data_normal.at[index, "SD_R_2_Ohm/km"],
                                                         data_normal.at[index, "SD_L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "SD_C_2_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "SD_Gama2_val"] = abs(aux)
    data_normal.at[index, "SD_Gama2_angle"] = np.angle(aux)      
    aux = Complex_Propagation_Constant(data_normal.at[index, "SD_R_3_Ohm/km"],
                                                         data_normal.at[index, "SD_L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "SD_C_3_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "SD_Gama3_val"] = abs(aux)
    data_normal.at[index, "SD_Gama3_angle"] = np.angle(aux)        
    
    
    aux = Complex_Propagation_Constant(data_normal.at[index, "SH_R_1_Ohm/km"],
                                                         data_normal.at[index, "SH_L_1_H/km"],
                                                         0,
                                                         data_normal.at[index, "SH_C_1_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "SH_Gama1_val"] = abs(aux)
    data_normal.at[index, "SH_Gama1_angle"] = np.angle(aux)    
    aux = Complex_Propagation_Constant(data_normal.at[index, "SH_R_2_Ohm/km"],
                                                         data_normal.at[index, "SH_L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "SH_C_2_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "SH_Gama2_val"] = abs(aux)
    data_normal.at[index, "SH_Gama2_angle"] = np.angle(aux)      
    aux = Complex_Propagation_Constant(data_normal.at[index, "SH_R_3_Ohm/km"],
                                                         data_normal.at[index, "SH_L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "SH_C_3_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "SH_Gama3_val"] = abs(aux)
    data_normal.at[index, "SH_Gama3_angle"] = np.angle(aux)     

    aux = Complex_Characteristic_Impedance(data_normal.at[index, "val_R_1_Ohm/km"],
                                                     data_normal.at[index, "val_L_1_H/km"],
                                                     0,
                                                     data_normal.at[index, "val_C_1_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "val_Z1_val"] = abs(aux)
    data_normal.at[index, "val_Z1_angle"] = np.angle(aux)  
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "val_R_2_Ohm/km"],
                                                     data_normal.at[index, "val_L_2_H/km"],
                                                     0,
                                                     data_normal.at[index, "val_C_2_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "val_Z2_val"] = abs(aux)
    data_normal.at[index, "val_Z2_angle"] = np.angle(aux) 
    aux = Complex_Characteristic_Impedance(data_normal.at[index, "val_R_3_Ohm/km"],
                                                     data_normal.at[index, "val_L_3_H/km"],
                                                     0,
                                                     data_normal.at[index, "val_C_3_F/km"],
                                                     2 * math.pi * 50)
    data_normal.at[index, "val_Z3_val"] = abs(aux)
    data_normal.at[index, "val_Z3_angle"] = np.angle(aux)   
    aux = Complex_Propagation_Constant(data_normal.at[index, "val_R_1_Ohm/km"],
                                                         data_normal.at[index, "val_L_1_H/km"],
                                                         0,
                                                         data_normal.at[index, "val_C_1_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "val_Gama1_val"] = abs(aux)
    data_normal.at[index, "val_Gama1_angle"] = np.angle(aux)     
    aux = Complex_Propagation_Constant(data_normal.at[index, "val_R_2_Ohm/km"],
                                                         data_normal.at[index, "val_L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "val_C_2_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "val_Gama2_val"] = abs(aux)
    data_normal.at[index, "val_Gama2_angle"] = np.angle(aux)     
    aux = Complex_Propagation_Constant(data_normal.at[index, "val_R_3_Ohm/km"],
                                                         data_normal.at[index, "val_L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "val_C_3_F/km"],
                                                         2 * math.pi * 50)
    data_normal.at[index, "val_Gama3_val"] = abs(aux)
    data_normal.at[index, "val_Gama3_angle"] = np.angle(aux)     

In [ ]:

data_normal["Z1_val"] = 0
data_normal["Z1_angle"] = 0
data_normal["Z2_val"] = 0
data_normal["Z2_angle"] = 0
data_normal["Z3_val"] = 0
data_normal["Z3_angle"] = 0
data_normal["Gama1_val"] = 0
data_normal["Gama1_angle"] = 0
data_normal["Gama2_val"] = 0
data_normal["Gama2_angle"] = 0
data_normal["Gama3_val"] = 0
data_normal["Gama3_angle"] = 0
for index in data_normal.index.to_list():
    
    
    data_normal.at[index, "Z1_val"] = abs(Complex_Characteristic_Impedance(data_normal.at[index, "R_1_Ohm/km"],
                                                     data_normal.at[index, "L_1_H/km"],
                                                     0,
                                                     data_normal.at[index, "C_1_F/km"],
                                                     2 * math.pi * 50))
    data_normal.at[index, "Z1_angle"] = np.angle(Complex_Characteristic_Impedance(data_normal.at[index, "R_1_Ohm/km"],
                                                         data_normal.at[index, "L_1_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_1_F/km"],
                                                         2 * math.pi * 50))
    data_normal.at[index, "Z2_val"] = abs(Complex_Characteristic_Impedance(data_normal.at[index, "R_2_Ohm/km"],
                                                         data_normal.at[index, "L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_2_F/km"],
                                                         2 * math.pi * 50))   
    data_normal.at[index, "Z2_angle"] = np.angle(Complex_Characteristic_Impedance(data_normal.at[index, "R_2_Ohm/km"],
                                                         data_normal.at[index, "L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_2_F/km"],
                                                         2 * math.pi * 50)) 
    data_normal.at[index, "Z3_val"] = abs(Complex_Characteristic_Impedance(data_normal.at[index, "R_3_Ohm/km"],
                                                         data_normal.at[index, "L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_3_F/km"],
                                                         2 * math.pi * 50))
    data_normal.at[index, "Z3_angle"] = np.angle(Complex_Characteristic_Impedance(data_normal.at[index, "R_3_Ohm/km"],
                                                         data_normal.at[index, "L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_3_F/km"],
                                                         2 * math.pi * 50))
    data_normal.at[index, "Gama1_val"] = abs(Complex_Propagation_Constant(data_normal.at[index, "R_1_Ohm/km"],
                                                         data_normal.at[index, "L_1_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_1_F/km"],
                                                         2 * math.pi * 50))
    data_normal.at[index, "Gama1_angle"] = np.angle(Complex_Propagation_Constant(data_normal.at[index, "R_1_Ohm/km"],
                                                         data_normal.at[index, "L_1_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_1_F/km"],
                                                         2 * math.pi * 50))
    data_normal.at[index, "Gama2_val"] = abs(Complex_Propagation_Constant(data_normal.at[index, "R_2_Ohm/km"],
                                                         data_normal.at[index, "L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_2_F/km"],
                                                         2 * math.pi * 50))

    data_normal.at[index, "Gama2_angle"] = np.angle(Complex_Propagation_Constant(data_normal.at[index, "R_2_Ohm/km"],
                                                         data_normal.at[index, "L_2_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_2_F/km"],
                                                         2 * math.pi * 50))

    data_normal.at[index, "Gama3_val"] = abs(Complex_Propagation_Constant(data_normal.at[index, "R_3_Ohm/km"],
                                                         data_normal.at[index, "L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_3_F/km"],
                                                         2 * math.pi * 50))

    data_normal.at[index, "Gama3_angle"] = np.angle(Complex_Propagation_Constant(data_normal.at[index, "R_3_Ohm/km"],
                                                         data_normal.at[index, "L_3_H/km"],
                                                         0,
                                                         data_normal.at[index, "C_3_F/km"],
                                                         2 * math.pi * 50))
   

In [ ]:
data_normal.to_csv("data_normal_recompute.csv")

In [ ]:
list_columns_analyze = [
'ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 'cycle', 
 'Iabc_generator_FirstArmonic_PhaseA_Value', 'Iabc_generator_FirstArmonic_PhaseA_Angle',
 'Iabc_generator_FirstArmonic_PhaseB_Value', 'Iabc_generator_FirstArmonic_PhaseB_Angle',
 'Iabc_generator_FirstArmonic_PhaseC_Value', 'Iabc_generator_FirstArmonic_PhaseC_Angle',
 'Vabc_generator_FirstArmonic_PhaseA_Value', 'Vabc_generator_FirstArmonic_PhaseA_Angle',
 'Vabc_generator_FirstArmonic_PhaseB_Value', 'Vabc_generator_FirstArmonic_PhaseB_Angle',
 'Vabc_generator_FirstArmonic_PhaseC_Value', 'Vabc_generator_FirstArmonic_PhaseC_Angle',
 'Iabc_consumator_FirstArmonic_PhaseA_Value', 'Iabc_consumator_FirstArmonic_PhaseA_Angle',
 'Iabc_consumator_FirstArmonic_PhaseB_Value', 'Iabc_consumator_FirstArmonic_PhaseB_Angle',
 'Iabc_consumator_FirstArmonic_PhaseC_Value', 'Iabc_consumator_FirstArmonic_PhaseC_Angle',    
 'Vabc_consumator_FirstArmonic_PhaseA_Value', 'Vabc_consumator_FirstArmonic_PhaseA_Angle',
 'Vabc_consumator_FirstArmonic_PhaseB_Value', 'Vabc_consumator_FirstArmonic_PhaseB_Angle',
 'Vabc_consumator_FirstArmonic_PhaseC_Value', 'Vabc_consumator_FirstArmonic_PhaseC_Angle',    
 'L_1_H/km', 'L_2_H/km', 'L_3_H/km', 'C_1_F/km', 'C_2_F/km', 'C_3_F/km', 'R_1_Ohm/km', 'R_2_Ohm/km', 'R_3_Ohm/km',
 'Z1_val', 'Z1_angle', 'Z2_val', 'Z2_angle', 'Z3_val', 'Z3_angle',
 'Gama1_val', 'Gama1_angle', 'Gama2_val', 'Gama2_angle', 'Gama3_val', 'Gama3_angle' ]

In [ ]:
ListSim = list(data_normal['ID_Simulare'].unique())

In [ ]:
file_name = 'val_transmissionWaves_Halley_Methode_F1.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               'A_10','B_10','C_10' ,'Err_A_10','Err_B_10','Err_C_10',
               'A_11','B_11','C_11', 'Err_A_11','Err_B_11','Err_C_11', 
               'A_12','B_12','C_12', 'Err_A_12','Err_B_12','Err_C_12', 
               'A_13','B_13','C_13', 'Err_A_13','Err_B_13','Err_C_13',
               'A_14','B_14','C_14', 'Err_A_14','Err_B_14','Err_C_14',
               'A_15','B_15','C_15', 'Err_A_15','Err_B_15','Err_C_15',
               'A_16','B_16','C_16', 'Err_A_16','Err_B_16','Err_C_16',
               'A_17','B_17','C_17', 'Err_A_17','Err_B_17','Err_C_17',
               'A_18','B_18','C_18', 'Err_A_18','Err_B_18','Err_C_18', 
               'A_19','B_19','C_19', 'Err_A_19','Err_B_19','Err_C_19' 
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            _Z_1_Compl = Create_Complex(float(_data['val_Z1_val']),
                                       float(_data['val_Z1_angle']),inp='rad')
            _Z_2_Compl = Create_Complex(float(_data['val_Z2_val']),
                                       float(_data['val_Z2_angle']),inp='rad')
            _Z_3_Compl = Create_Complex(float(_data['val_Z3_val']),
                                       float(_data['val_Z3_angle']),inp='rad')

            _gama_1_Compl = Create_Complex(float(_data['val_Gama1_val']),
                                       float(_data['val_Gama1_angle']),inp='rad')
            _gama_2_Compl = Create_Complex(float(_data['val_Gama2_val']),
                                       float(_data['val_Gama2_angle']),inp='rad')
            _gama_3_Compl = Create_Complex(float(_data['val_Gama3_val']),
                                       float(_data['val_Gama3_angle']),inp='rad')

            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')
            
            _transmissionWaves1 = Transmission_Waves(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                                    float(_data['TotalLength']), _gama_1_Compl, _Z_1_Compl)

            _transmissionWaves2 = Transmission_Waves(_U_B_1_Compl, _U_B_2_Compl, _I_B_1_Compl, _I_B_2_Compl, 
                                                    float(_data['TotalLength']), _gama_2_Compl, _Z_2_Compl)

            _transmissionWaves3 = Transmission_Waves(_U_C_1_Compl, _U_C_2_Compl, _I_C_1_Compl, _I_C_2_Compl, 
                                                    float(_data['TotalLength']), _gama_3_Compl, _Z_3_Compl)
    
            aux.append(abs(_transmissionWaves1.Halley_Methode_F1(1000))*Total_Length)
            aux.append(abs(_transmissionWaves2.Halley_Methode_F1(1000))*Total_Length)
            aux.append(abs(_transmissionWaves3.Halley_Methode_F1(1000))*Total_Length)
            aux.append(abs(fault_value - abs(_transmissionWaves1.Halley_Methode_F1(1000))*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - abs(_transmissionWaves2.Halley_Methode_F1(1000))*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - abs(_transmissionWaves3.Halley_Methode_F1(1000))*Total_Length)*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))

In [ ]:
file_name = 'val_cuadripol_Determin_Minim_F1.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               'A_10','B_10','C_10' ,'Err_A_10','Err_B_10','Err_C_10',
               'A_11','B_11','C_11', 'Err_A_11','Err_B_11','Err_C_11', 
               'A_12','B_12','C_12', 'Err_A_12','Err_B_12','Err_C_12', 
               'A_13','B_13','C_13', 'Err_A_13','Err_B_13','Err_C_13',
               'A_14','B_14','C_14', 'Err_A_14','Err_B_14','Err_C_14',
               'A_15','B_15','C_15', 'Err_A_15','Err_B_15','Err_C_15',
               'A_16','B_16','C_16', 'Err_A_16','Err_B_16','Err_C_16',
               'A_17','B_17','C_17', 'Err_A_17','Err_B_17','Err_C_17',
               'A_18','B_18','C_18', 'Err_A_18','Err_B_18','Err_C_18', 
               'A_19','B_19','C_19', 'Err_A_19','Err_B_19','Err_C_19' 
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            
            prop_cuad1 = Cuadripols( float(_data['val_R_1_Ohm/km'])
                                    ,float(_data['val_L_1_H/km'])
                                    ,0
                                    ,float(_data['val_C_1_F/km'])
                                    ,2*math.pi*50)
            prop_cuad2 = Cuadripols( float(_data['val_R_2_Ohm/km'])
                                    ,float(_data['val_L_2_H/km'])
                                    ,0
                                    ,float(_data['val_C_2_F/km'])
                                    ,2*math.pi*50)            
            
            prop_cuad3 = Cuadripols( float(_data['val_R_3_Ohm/km'])
                                    ,float(_data['val_L_3_H/km'])
                                    ,0
                                    ,float(_data['val_C_3_F/km'])
                                    ,2*math.pi*50) 
            
            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')      
            
            _cuad1 = Cuadripol_Method(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                      Total_Length, prop_cuad1.matrix, 0.001)
            _cuad2 = Cuadripol_Method(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                      Total_Length, prop_cuad1.matrix, 0.001)
            _cuad3 = Cuadripol_Method(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                      Total_Length, prop_cuad1.matrix, 0.001)            
            aux.append(_cuad1.Determin_Minim_F1(300)*Total_Length)
            aux.append(_cuad2.Determin_Minim_F1(300)*Total_Length)
            aux.append(_cuad3.Determin_Minim_F1(300)*Total_Length)
            
            aux.append(abs(fault_value - _cuad1.Determin_Minim_F1(300)*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - _cuad2.Determin_Minim_F1(300)*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - _cuad3.Determin_Minim_F1(300)*Total_Length)*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))

In [ ]:
file_name = '_transmissionWaves_Determin_Minim_F2.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               'A_10','B_10','C_10' ,'Err_A_10','Err_B_10','Err_C_10',
               'A_11','B_11','C_11', 'Err_A_11','Err_B_11','Err_C_11', 
               'A_12','B_12','C_12', 'Err_A_12','Err_B_12','Err_C_12', 
               'A_13','B_13','C_13', 'Err_A_13','Err_B_13','Err_C_13',
               'A_14','B_14','C_14', 'Err_A_14','Err_B_14','Err_C_14',
               'A_15','B_15','C_15', 'Err_A_15','Err_B_15','Err_C_15',
               'A_16','B_16','C_16', 'Err_A_16','Err_B_16','Err_C_16',
               'A_17','B_17','C_17', 'Err_A_17','Err_B_17','Err_C_17',
               'A_18','B_18','C_18', 'Err_A_18','Err_B_18','Err_C_18', 
               'A_19','B_19','C_19', 'Err_A_19','Err_B_19','Err_C_19' 
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            _Z_1_Compl = Create_Complex(float(_data['Z1_val']),
                                       float(_data['Z1_angle']),inp='rad')
            _Z_2_Compl = Create_Complex(float(_data['Z2_val']),
                                       float(_data['Z2_angle']),inp='rad')
            _Z_3_Compl = Create_Complex(float(_data['Z3_val']),
                                       float(_data['Z3_angle']),inp='rad')

            _gama_1_Compl = Create_Complex(float(_data['Gama1_val']),
                                       float(_data['Gama1_angle']),inp='rad')
            _gama_2_Compl = Create_Complex(float(_data['Gama2_val']),
                                       float(_data['Gama2_angle']),inp='rad')
            _gama_3_Compl = Create_Complex(float(_data['Gama3_val']),
                                       float(_data['Gama3_angle']),inp='rad')

            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')
            _transmissionWaves1 = Transmission_Waves(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                                    float(_data['TotalLength']), _gama_1_Compl, _Z_1_Compl)

            _transmissionWaves2 = Transmission_Waves(_U_B_1_Compl, _U_B_2_Compl, _I_B_1_Compl, _I_B_2_Compl, 
                                                    float(_data['TotalLength']), _gama_2_Compl, _Z_2_Compl)

            _transmissionWaves3 = Transmission_Waves(_U_C_1_Compl, _U_C_2_Compl, _I_C_1_Compl, _I_C_2_Compl, 
                                                    float(_data['TotalLength']), _gama_3_Compl, _Z_3_Compl)
    
            aux.append(abs(_transmissionWaves1.Determin_Minim_F1(1000))*Total_Length)
            aux.append(abs(_transmissionWaves2.Determin_Minim_F1(1000))*Total_Length)
            aux.append(abs(_transmissionWaves3.Determin_Minim_F1(1000))*Total_Length)
            aux.append(abs(fault_value - abs(_transmissionWaves1.Determin_Minim_F1(1000))*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - abs(_transmissionWaves2.Determin_Minim_F1(1000))*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - abs(_transmissionWaves3.Determin_Minim_F1(1000))*Total_Length)*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))

In [ ]:
prop_cuad1 = Cuadripols(1,1,1,1,50)

In [ ]:
prop_cuad1.matrix

In [ ]:
file_name = 'cuadripol_Determin_Minim_F2.csv'
result = []
result.append(['ID_Simulare', 'obs_Simulare', 'Nume_Grup', 'Simulation_name', 'Type', 'phase',
 'FaultLocation_FromGenerator', 'FaultLocation_ToConsumer', 'TotalLength', 
               'A_10','B_10','C_10' ,'Err_A_10','Err_B_10','Err_C_10',
               'A_11','B_11','C_11', 'Err_A_11','Err_B_11','Err_C_11', 
               'A_12','B_12','C_12', 'Err_A_12','Err_B_12','Err_C_12', 
               'A_13','B_13','C_13', 'Err_A_13','Err_B_13','Err_C_13',
               'A_14','B_14','C_14', 'Err_A_14','Err_B_14','Err_C_14',
               'A_15','B_15','C_15', 'Err_A_15','Err_B_15','Err_C_15',
               'A_16','B_16','C_16', 'Err_A_16','Err_B_16','Err_C_16',
               'A_17','B_17','C_17', 'Err_A_17','Err_B_17','Err_C_17',
               'A_18','B_18','C_18', 'Err_A_18','Err_B_18','Err_C_18', 
               'A_19','B_19','C_19', 'Err_A_19','Err_B_19','Err_C_19' 
              ])
for _l in ListSim:
    Total_Length = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['TotalLength']) 
    fault_value = float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_FromGenerator'])
    aux = [int(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['ID_Simulare']),
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['obs_Simulare'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Nume_Grup'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Simulation_name'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['Type'])[0],
           list(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['phase'])[0],
           fault_value,
           float(data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == _l)]['FaultLocation_ToConsumer']),
           Total_Length
          ]
    for _cycle in range(10,20):
        _data = data_normal[(data_normal['cycle'] == _cycle) & (data_normal['ID_Simulare'] == _l)]
        if (len(_data)>0):
            
            prop_cuad1 = Cuadripols( float(_data['R_1_Ohm/km'])
                                    ,float(_data['L_1_H/km'])
                                    ,0
                                    ,float(_data['C_1_F/km'])
                                    ,2*math.pi*50)
            prop_cuad2 = Cuadripols( float(_data['R_2_Ohm/km'])
                                    ,float(_data['L_2_H/km'])
                                    ,0
                                    ,float(_data['C_2_F/km'])
                                    ,2*math.pi*50)            
            
            prop_cuad3 = Cuadripols( float(_data['R_3_Ohm/km'])
                                    ,float(_data['L_3_H/km'])
                                    ,0
                                    ,float(_data['C_3_F/km'])
                                    ,2*math.pi*50) 
            
            _U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _U_B_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _U_C_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Vabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

            _I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseA_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
            _I_B_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseB_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
            _I_C_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseC_Value']),
                                       float(_data['Iabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')      
            
            _cuad1 = Cuadripol_Method(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                      Total_Length, prop_cuad1.matrix, 0.001)
            _cuad2 = Cuadripol_Method(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                      Total_Length, prop_cuad1.matrix, 0.001)
            _cuad3 = Cuadripol_Method(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                      Total_Length, prop_cuad1.matrix, 0.001)            
            aux.append(_cuad1.Determin_Minim_F2(1000)*Total_Length)
            aux.append(_cuad2.Determin_Minim_F2(1000)*Total_Length)
            aux.append(_cuad3.Determin_Minim_F2(1000)*Total_Length)
            
            aux.append(abs(fault_value - _cuad1.Determin_Minim_F2(1000)*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - _cuad2.Determin_Minim_F2(1000)*Total_Length)*100/fault_value)
            aux.append(abs(fault_value - _cuad3.Determin_Minim_F2(1000)*Total_Length)*100/fault_value)
                         
    result.append(aux)
result_dataFrame=pd.DataFrame.from_records(result[1:],columns=result[0])
result_dataFrame.to_csv(os.path.join(os.getcwd(),file_name))

In [ ]:
2*math.pi*50

In [ ]:
data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == 1)]['obs_Simulare']


In [ ]:
_data = data_normal[(data_normal['cycle'] == 10) & (data_normal['ID_Simulare'] == 1)]

In [ ]:
_Z_1_Compl = Create_Complex(float(_data['Z1_val']),
                           float(_data['Z1_angle']),inp='rad')
_Z_2_Compl = Create_Complex(float(_data['Z2_val']),
                           float(_data['Z2_angle']),inp='rad')
_Z_3_Compl = Create_Complex(float(_data['Z3_val']),
                           float(_data['Z3_angle']),inp='rad')

_gama_1_Compl = Create_Complex(float(_data['Gama1_val']),
                           float(_data['Gama1_angle']),inp='rad')
_gama_2_Compl = Create_Complex(float(_data['Gama2_val']),
                           float(_data['Gama2_angle']),inp='rad')
_gama_3_Compl = Create_Complex(float(_data['Gama3_val']),
                           float(_data['Gama3_angle']),inp='rad')

_U_A_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseA_Value']),
                           float(_data['Vabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
_U_B_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseB_Value']),
                           float(_data['Vabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
_U_C_1_Compl = Create_Complex(float(_data['Vabc_generator_FirstArmonic_PhaseC_Value']),
                           float(_data['Vabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

_U_A_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseA_Value']),
                           float(_data['Vabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
_U_B_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseB_Value']),
                           float(_data['Vabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
_U_C_2_Compl = Create_Complex(float(_data['Vabc_consumator_FirstArmonic_PhaseC_Value']),
                           float(_data['Vabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')

_I_A_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseA_Value']),
                           float(_data['Iabc_generator_FirstArmonic_PhaseA_Angle']),inp='rad')
_I_B_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseB_Value']),
                           float(_data['Iabc_generator_FirstArmonic_PhaseB_Angle']),inp='rad')
_I_C_1_Compl = Create_Complex(float(_data['Iabc_generator_FirstArmonic_PhaseC_Value']),
                           float(_data['Iabc_generator_FirstArmonic_PhaseC_Angle']),inp='rad')

_I_A_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseA_Value']),
                           float(_data['Iabc_consumator_FirstArmonic_PhaseA_Angle']),inp='rad')
_I_B_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseB_Value']),
                           float(_data['Iabc_consumator_FirstArmonic_PhaseB_Angle']),inp='rad')
_I_C_2_Compl = Create_Complex(float(_data['Iabc_consumator_FirstArmonic_PhaseC_Value']),
                           float(_data['Iabc_consumator_FirstArmonic_PhaseC_Angle']),inp='rad')

In [ ]:
_transmissionWaves1 = Transmission_Waves(_U_A_1_Compl, _U_A_2_Compl, _I_A_1_Compl, _I_A_2_Compl, 
                                        float(_data['TotalLength']), _gama_1_Compl, _Z_1_Compl)

_transmissionWaves2 = Transmission_Waves(_U_B_1_Compl, _U_B_2_Compl, _I_B_1_Compl, _I_B_2_Compl, 
                                        float(_data['TotalLength']), _gama_2_Compl, _Z_2_Compl)

_transmissionWaves3 = Transmission_Waves(_U_C_1_Compl, _U_C_2_Compl, _I_C_1_Compl, _I_C_2_Compl, 
                                        float(_data['TotalLength']), _gama_3_Compl, _Z_3_Compl)

In [ ]:
_transmissionWaves1.Compute_F1(1)

In [ ]:
_U_A_1_Compl * cmath.cosh(_gama_1_Compl * float(_data['TotalLength']) * 1) 
- _U_A_2_Compl * cmath.cosh(_gama_1_Compl * float(_data['TotalLength']) * (1-1))

In [ ]:
abs(_transmissionWaves1.Newton_Methode_F1())

In [ ]:
abs(_transmissionWaves1.Newton_Methode_F2(2000))

In [ ]:
abs(_transmissionWaves1.Halley_Methode_F1(20000))

In [ ]:
abs(_transmissionWaves1.Halley_Methode_F2(20000))

In [ ]:
_transmissionWaves1.Determin_Minim_F1_list(1000)